In [185]:
# This python notebook annotate the bioconda recipes using the ids from bio.tools in the external session 
# in bioconda recipes. Two different input should be provided, the bioconda_recipes_path in your local machine and
# the file with the mapping.   


import glob2
import os

import requests
import sys
import yaml
import getopt
import re
from pandas import DataFrame, read_csv

from ruamel.yaml import YAML

from yaml.constructor import ConstructorError
from yaml.scanner import ScannerError

In [190]:
def get_bioconda_yalm(bioconda_path):
    yalm_files = dict()
    matches =  glob2.glob(bioconda_path + '/**/*.yaml', with_matches=True)
    yaml = YAML(typ='jinja2')
    yaml.allow_duplicate_keys = True
    yaml.allow_unicode = True
    if(len(matches) > 0 ):
        for file_name in matches:
            if(os.path.isfile(file_name[0])):
                #print(file_name[0])
                path_list = file_name[0].replace(bioconda_path, "").split(os.sep)
                software_name = path_list[1]
                try:
                    with open(file_name[0]) as fp:
                         yalm_file = yaml.load(fp)
                    yalm_files[software_name] = (file_name[0], yalm_file)
                except (ScannerError, ConstructorError, TypeError, AttributeError) as error:
                    print("ERROR | " + software_name + "\t" + file_name[0])
    return yalm_files          

In [191]:
def get_annotation_mapping(mapping_file):
    mapping = read_csv(mapping_file)
    return mapping    

In [192]:
def annotate_yaml(yaml_file, biotool):
    path_file = yaml_file[0]
    yaml_file_content = yaml_file[1]
    if yaml_file_content.get('extra') is None:
        yaml_file_content['extra'] = {}
    else:
        print("EXTRA has been found: " + biotool)
    yaml_file_content['extra']['biotools']=[biotool]
    with open(path_file, 'w') as fp:
        yaml.dump(yaml_file_content, fp)
    

def annotate_yamls(yaml_dictonary, mappings):
    for (idx, row) in mappings.iterrows():
        if yaml_dictonary.get(row.bioconda) is not None:
            annotate_yaml(yaml_dictonary[row.bioconda], row.biotools)

In [193]:
bioconda_recipes_path = "/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes"
mapping_file          = "/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-biotools/mapping_matchonly.csv"
mapping = get_annotation_mapping(mapping_file)
yaml_files = get_bioconda_yalm(bioconda_recipes_path)
print(yaml_files)
annotate_yamls(yaml_files, mapping)

{'2pg_cartesian': ('/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/2pg_cartesian/meta.yaml', CommentedMap([('about', CommentedMap([('home', 'https://github.com/rodrigofaccioli/2pg_cartesian'), ('license_file', 'LICENSE'), ('summary', '2pg cartesian is a framework of optimization algorithms for protein structure prediction.')])), ('package', CommentedMap([('name', '2pg_cartesian'), ('version', '1.0.1')])), ('build', CommentedMap([('skip', True), ('number', 0)])), ('source', CommentedMap([('fn', 'v1.0.1.tar.gz'), ('url', 'https://github.com/rodrigofaccioli/2pg_cartesian/archive/v1.0.1.tar.gz'), ('md5', 'd97fcd5fff5cb0813d4ea918c57d991f')])), ('requirements', CommentedMap([('build', ['gcc', 'llvm', 'cmake']), ('run', ['libgcc', 'gromacs'])])), ('test', CommentedMap([('commands', ['protpred-Gromacs-Test_random_number'])]))])), 'abawaca': ('/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/abawaca/meta.yaml', CommentedMap([('package', Comme

EXTRA has been found: bbmap
EXTRA has been found: freec
EXTRA has been found: searchgui
EXTRA has been found: trinity
EXTRA has been found: chromhmm
EXTRA has been found: fraggenescan
EXTRA has been found: gatk
EXTRA has been found: tripal
EXTRA has been found: openms
EXTRA has been found: freebayes
EXTRA has been found: vep
EXTRA has been found: sff2fastq
EXTRA has been found: qiime
EXTRA has been found: effectivet3
EXTRA has been found: mixomics
EXTRA has been found: Novoalign
EXTRA has been found: multiqc


{'meta.yaml': CommentedMap([('package', CommentedMap([('name', 'zifa'), ('version', '0.1.0')])), ('source', CommentedMap([('fn', 'zifa-0.1.0.zip'), ('url', 'https://github.com/epierson9/ZIFA/archive/e40fdbf6c41cd7e564ab03d242fc071f5ac543b7.zip')])), ('build', CommentedMap([('number', 0)])), ('requirements', CommentedMap([('build', ['python', 'setuptools']), ('run', ['python'])])), ('test', CommentedMap([('imports', ['ZIFA'])])), ('about', CommentedMap([('home', 'https://github.com/epierson9/ZIFA'), ('license', 'MIT'), ('summary', 'Dimensionality reduction for zero-inflated single-cell gene expression analysis')]))]), 'biotools.yaml': CommentedMap([('function', [CommentedMap([('input', [CommentedMap([('dataType', CommentedMap([('term', 'RNA sequence'), ('uri', 'http://edamontology.org/data_3495')])), ('dataFormat', [CommentedMap([('term', 'FASTA'), ('uri', 'http://edamontology.org/format_1929')]), CommentedMap([('term', 'FASTA-like (text)'), ('uri', 'http://edamontology.org/format_2200'